In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: machinelearning
Azure region: germanywestcentral
Subscription id: 4999417e-f032-4d4c-982a-c229e26aa825
Resource group: udacity-learning


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

cluster_name = "standardcluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    cluster_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(-0.01, 1.0),
    '--max_iter': choice(50, 100, 150)
}) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='./',
    script='train.py',
    compute_target=cluster,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=12,
    max_concurrent_runs=4
)

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdriver_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdriver_run).show()
hyperdriver_run.get_status()
hyperdriver_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_1a0a21f3-6990-4ff3-ad44-1ffa5d33f59d
Web View: https://ml.azure.com/runs/HD_1a0a21f3-6990-4ff3-ad44-1ffa5d33f59d?wsid=/subscriptions/4999417e-f032-4d4c-982a-c229e26aa825/resourcegroups/udacity-learning/workspaces/machinelearning&tid=fd3b6086-53e5-43ac-9b6b-b17e80f6e2f7

Streaming azureml-logs/hyperdrive.txt

[2022-09-16T12:44:53.770392][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-09-16T12:44:54.7075847Z][SCHEDULER][INFO]Scheduling job, id='HD_1a0a21f3-6990-4ff3-ad44-1ffa5d33f59d_0' 
[2022-09-16T12:44:54.7880991Z][SCHEDULER][INFO]Scheduling job, id='HD_1a0a21f3-6990-4ff3-ad44-1ffa5d33f59d_1' 
[2022-09-16T12:44:54.9111792Z][SCHEDULER][INFO]Scheduling job, id='HD_1a0a21f3-6990-4ff3-ad44-1ffa5d33f59d_2' 
[2022-09-16T12:44:55.0241869Z][SCHEDULER][INFO]Scheduling job, id='HD_1a0a21f3-6990-4ff3-ad44-1ffa5d33f59d_3' 
[2022-09-16T12:44:54.955988][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-

{'runId': 'HD_1a0a21f3-6990-4ff3-ad44-1ffa5d33f59d',
 'target': 'standardcluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-16T12:44:52.931283Z',
 'endTimeUtc': '2022-09-16T13:05:00.149084Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '95f5e755-e051-4b96-9711-056882b4fef3',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': 'infinite_space_size',
  'score': '0.9116240573845871',
  'best_child_run_id': 'HD_1a0a21f3-6990-4ff3-ad44-1ffa5d33f59d_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_1a0a21f3-6990-4ff3-ad44-1ffa5d33f59d_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetry

KeyError: 'log_files'

In [29]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run_metric = hyperdriver_run.get_best_run_by_primary_metric()
best_metrics = best_run_metric.get_metrics()
print('\nAccuracy:', best_metrics['Accuracy'])
print(best_run_metric.get_file_names())
print('Best Run: ', best_run_metric)
model = best_run_metric.register_model(model_name='best_run_metric', model_path='/')


Accuracy: 0.9116240573845871
['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2022-09-16-12', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']
Best Run:  Run(Experiment: udacity-project,
Id: HD_1a0a21f3-6990-4ff3-ad44-1ffa5d33f59d_1,
Type: azureml.scriptrun,
Status: Completed)


In [32]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_url)

In [33]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [34]:
y

0        0
1        0
2        0
3        0
4        0
        ..
32945    0
32946    0
32947    0
32948    0
32949    0
Name: y, Length: 32950, dtype: int64

In [40]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.


automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target=cluster,
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    max_concurrent_iterations=4,
    n_cross_validations=4)

In [41]:
# Submit your automl run

### YOUR CODE HERE ###

exp = Experiment(ws, "udacity-project")
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting remote run.
No run_configuration provided, running on standardcluster with default configuration
Running on remote compute: standardcluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_1b1a50b1-54cb-44dc-b67d-94a2eb4715a8,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_1b1a50b1-54cb-44dc-b67d-94a2eb4715a8,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |yes                             |32950                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_1b1a50b1-54cb-44dc-b67d-94a2eb4715a8',
 'target': 'standardcluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-16T13:28:26.519415Z',
 'endTimeUtc': '2022-09-16T14:14:38.200933Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'standardcluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"4999417e-f032-4d4c-982a-c229e26aa825","resource_group":"udacity-learning","workspace_name":"machinelearning","region":"germanywestcentral","compute_target":"standardcluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":

In [42]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, best_automl_model = automl_run.get_output()
print(f"Best run : {best_run}")
print(f"Best model : {best_automl_model}")

print(f"Best model name : {best_run.properties['model_name']}")

best_automl_model = automl_run.register_model(model_name='best_automl_model')

Best run : Run(Experiment: udacity-project,
Id: AutoML_1b1a50b1-54cb-44dc-b67d-94a2eb4715a8_30,
Type: azureml.scriptrun,
Status: Completed)
Best model : Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('25', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.4955555555555555, learning_rate=0.05789894736842106, max_bin=210, max_depth=5, min_child_weight=0, min_data_in_leaf=0.07931241379310346, min_split_